## ABC Simple Installation System: Post-processing Upgrade Results
(savings relative to thermal loads) \
Created on: 10/12/2021 \
By: Lixi Liu (Lixi.Liu@nrel.gov)

Note: change kernel to that of EULP Calibration & Validation before running

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
print(f'Notebook path: {os.getcwd()}')
from pathlib import Path
import eulpcv.resstock_enduse_categories as res_enduse_categories

### Download results online
* unprocessed upgrade results: S3/resbldg-datasets/abc-sis

### Initialize

In [ ]:
# local path to downloaded results
pkg_version = f'packages_v01' # <----- v01 (N=10), v02 (N=5)
result_dir = Path('/Users/lliu2/Documents/ABC Simple Intall Sys/results')
result_path = result_dir / pkg_version / 'results'
resdir = result_dir / pkg_version / 'processed results'

## create folder for post-processed results:
outdir = result_dir / pkg_version / 'result summary'
plot_path = outdir / 'plots'

os.makedirs(resdir, exist_ok=True)
os.makedirs(outdir, exist_ok=True)
os.makedirs(plot_path, exist_ok=True)

print(f'Run results come from: \n   {result_path}')
print(f'Output directory: \n   {outdir}')

### Variables

In [ ]:
nupgrades_dict = {
    'packages_v01': 10,
    'packages_v02': 5,
}

njobs_dict = {
    'packages_v01': 400,
    'packages_v02': 200,
}

upgrade_name_correction_dict = {
    "Air Tight Envelope": "Airtight Envelope", #1 (v01)
    "Ultra Air Tight Envelope - Mec Vent": "Ultra-Airtight Envelope - Mech Vent", #2 (v01)
    "Ultra Air Tight Envelope - Rec Vent": "Ultra-Airtight Envelope - HRV/ERV", #3 (v01)
    "Air Tight Envelope + Roof Insulation": "Airtight Envelope + Roof", #4 (v01)
    "Ultra Air Tight Envelope - Mech Vent + Roof Insulation": "Ultra-Airtight Envelope - Mech Vent + Roof", #5 (V01)
    "Air Tight Envelope + MSHP": "Airtight Envelope + MSHP", #6 (v02, #1)
    "Ultra Air Tight Envelope - Mech Vent + MSHP": "Ultra-Airtight Envelope - Mech Vent + MSHP", #7 (v02, #2)
    "Ultra Air Tight Envelope - Rec Vent + MSHP": "Ultra-Airtight Envelope - HRV/ERV + MSHP", #8 (v02, #3)
    "Air Tight Envelope + HPWH": "Airtight Envelope + HPWH", #9 (v01)
    "Ultra Air Tight Envelope - Mech Vent + HPWH": "Ultra-Airtight Envelope - Mech Vent + HPWH", #10 (v01)
    "Air Tight Envelope + MSHP _HPWH": "Airtight Envelope + MSHP + HPWH", #11 (v02, #4)
    "Ultra Air Tight Envelope - Rec Vent + MSHP + HPWH": "Ultra-Airtight Envelope - HRV/ERV + MSHP + HPWH", #12 (v02, #5)
}

upgrade_rename_dict = {
    "Air Tight Envelope": "Env", #1 (v01)
    "Ultra Air Tight Envelope - Mec Vent": "UEnv-Mech", #2 (v01)
    "Ultra Air Tight Envelope - Rec Vent": "UEnv-Rec", #3 (v01)
    "Air Tight Envelope + Roof Insulation": "Env + Roof", #4 (v01)
    "Ultra Air Tight Envelope - Mech Vent + Roof Insulation": "UEnv-Mech + Roof", #5 (V01)
    "Air Tight Envelope + MSHP": "Env + MSHP", #6 (v02, #1)
    "Ultra Air Tight Envelope - Mech Vent + MSHP": "UEnv-Mech + MSHP", #7 (v02, #2)
    "Ultra Air Tight Envelope - Rec Vent + MSHP": "UEnv-Rec + MSHP", #8 (v02, #3)
    "Air Tight Envelope + HPWH": "Env + HPWH", #9 (v01)
    "Ultra Air Tight Envelope - Mech Vent + HPWH": "UEnv-Mech + HPWH", #10 (v01)
    "Air Tight Envelope + MSHP _HPWH": "Env + MSHP + HPWH", #11 (v02, #4)
    "Ultra Air Tight Envelope - Rec Vent + MSHP + HPWH": "UEnv-Rec + MSHP + HPWH", #12 (v02, #5)
}
print('"upgrade_rename_dict" created')


## Get thermal load end uses
# electric
thermal_eu_main = ['hot_water', 'heating', 'cooling', 'hvac_fan_pump']
eu_dict = res_enduse_categories.enduse_category_dict()
eu_df = pd.DataFrame.from_dict(eu_dict, orient='index', columns=['enduse']) # turn eu_dict to df
thermal_eu_df = eu_df.query('enduse in @thermal_eu_main')
thermal_eu_df.index = 'simulation_output_report.' + thermal_eu_df.index
thermal_eu = thermal_eu_df.index

# gas
thermal_eu_gas = ["natural_gas_heating_therm",
                  "natural_gas_water_systems_therm",]
thermal_eu_gas = list('simulation_output_report.' + x for x in thermal_eu_gas)

# others
thermal_eu_other = ["fuel_oil_heating_mbtu",
                    "fuel_oil_water_systems_mbtu",
                    "propane_heating_mbtu",
                    "propane_water_systems_mbtu",
                    "wood_heating_mbtu",]
thermal_eu_other = list('simulation_output_report.' + x for x in thermal_eu_other)

print('"thermal_eu(s)" defined')


# conversion multipliers
kwh_to_mbtu = 0.003412141633127942
therm_to_mbtu = 0.1

print('"conversion factors" defined')

### Functions

In [ ]:
def add_thermal_load_sqft_eui(df, ref):
    """
    ARG:
        ref (df): baseline df
    RETURN:
        df with added cols: 'sqft', 'gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_mbtupersqft'
    """
    res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    df['sqft'] = res['simulation_output_report.floor_area_conditioned_ft_2']
    
    df['thermal_gas_therm'] = df[thermal_eu_gas].sum(axis=1)
    df['thermal_elec_kwh'] = df[thermal_eu].sum(axis=1)
    df['thermal_other_mbtu'] = df[thermal_eu_other].sum(axis=1)
    df['thermal_site_mbtu'] = df['thermal_elec_kwh']*kwh_to_mbtu + df['thermal_gas_therm']*therm_to_mbtu + df['thermal_other_mbtu']
    
    df['gas_eui_thermpersqft'] = df['thermal_gas_therm'].divide(df['sqft']) # therm/sqft
    df['elec_eui_kwhpersqft'] = df['thermal_elec_kwh'].divide(df['sqft']) # kwh/sqft
    df['site_eui_mbtupersqft'] = df['thermal_site_mbtu'].divide(df['sqft']) # mbtu/sqft
    
    for col in ['sqft','gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_mbtupersqft']:
        df.loc[df['simulation_output_report.applicable']==False, col] = np.nan
    
    return df
print('func loaded: "add_thermal_load_sqft_eui"')


# fix 1ACH50 upgrade costs in UEnv-Mech & UEnv-Rec
pkg_to_fix = [
    "Ultra Air Tight Envelope - Mec Vent", #2 (v01)
    "Ultra Air Tight Envelope - Rec Vent", #3 (v01)
    "Ultra Air Tight Envelope - Mech Vent + Roof Insulation", #5 (V01)
    "Ultra Air Tight Envelope - Mech Vent + MSHP", #7 (v02, #2)
    "Ultra Air Tight Envelope - Rec Vent + MSHP", #8 (v02, #3)
    "Ultra Air Tight Envelope - Mech Vent + HPWH", #10 (v01)
    "Ultra Air Tight Envelope - Rec Vent + MSHP + HPWH", #12 (v02, #5)
]

# prev, all at 5.1 $/sqft
air_seal_cost = {
    "50 ACH50": 10.21,
    "40 ACH50": 8.17,
    "30 ACH50": 6.13,
}

def fix_upgrade_costs(df, ref):
    print('  fixing 1 ACH50 air sealing cost...')
    
    upcost_prev = df['simulation_output_report.upgrade_cost_usd'].copy()
    res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    
    res['air_seal_cost'] = res['build_existing_model.infiltration'].map(air_seal_cost)
    res['air_seal_cost'] = res['air_seal_cost'].fillna(5.1)

    df['simulation_output_report.upgrade_cost_usd'] += (res['air_seal_cost']-5.1)*\
        res['simulation_output_report.floor_area_conditioned_ft_2']
    df['simulation_output_report.upgrade_cost_usd']
    
    upcost_delta = df['simulation_output_report.upgrade_cost_usd'] - upcost_prev
    upcost_delta_avg = upcost_delta.mean()
    upcost_change_count = upcost_delta[upcost_delta>0].count()
    (df['simulation_output_report.upgrade_cost_usd'] - upcost_prev)
    print(f'  1 ACH50 air sealing cost increased for {upcost_change_count} simulations, by: {upcost_delta_avg}')
    
    return df
print('func loaded: "fix_upgrade_costs"')


### 1. BASELINE results

In [ ]:
res = pd.read_csv(result_path / 'results_up00.csv')

# (1) get sqft, gas/elec/site eui
res = add_thermal_load_sqft_eui(res, res)

# check
Nbldg = 111004
Njob = njobs_dict[pkg_version] # <--

jobs_missing = set(range(1,Njob)) - set(res.job_id.unique())
print(f'- {len(jobs_missing)} jobs missing: {jobs_missing}')
print(f'- {Nbldg-len(res)} buildings ({((Nbldg-len(res))/Nbldg*100):.2f}%)')

res

### 1.1. Check housing charateristics distributions in BASELINE

In [ ]:
proto_chars = ['build_existing_model.geometry_stories',
               'build_existing_model.geometry_wall_type',
               'build_existing_model.vintage_acs',
               'build_existing_model.hvac_cooling_type',
               'build_existing_model.hvac_heating_type_and_fuel',
               'build_existing_model.geometry_floor_area',
               'build_existing_model.census_division',
               'build_existing_model.building_america_climate_zone',
             ]

Ns = len(res.query('completed_status=="Success"')); N = len(res)
print(f'>>> ResStock - {pkg_version} - BASELINE result summary:\n')
print(f'  * {Ns} / {N} samples ran successfully, {N-Ns} failed, efficacy: {Ns/N:.1%} \n')

print('>>> Housing characteristics splits:\n')

Res_char = []
for i, char in enumerate(proto_chars,1):
    Nchar = res.groupby(char)['building_id'].count()
    Nchar = Nchar/Ns
    Nchar['N_failed'] = len(res[res[char].isnull()])
    print(f'  * [{i}] {Nchar}\n')
    
    ## append for export
    Nchar = Nchar.rename('fraction').to_frame()
    Nchar['housing_char'] = Nchar.index.name
    Res_char.append(Nchar)
    
Res_char = pd.concat(Res_char, axis=0)
Res_char.index.name = 'sub_char'
Res_char = Res_char.reset_index()
Res_char = Res_char[['housing_char','sub_char','fraction']]

Res_char.to_csv(outdir / 'baseline_housing_char_breakdown.csv', index=False)

In [ ]:
res[res['completed_status']=="Success"].to_csv(resdir / 'results_up00.csv', index=False)
print('baseline result processed and saved')

## 2B. PACKAGE summary ###

In [ ]:
### upgrade result processing funcs
def assign_utility_rates_to_upgrade(upkg_no, p, ref, HVAC_upgrades_rate_change, for_packages=False):
    
    res = ref.set_index('building_id')[['gas_rate', 'gas_fixed', 'gas_CO2_rate',
                                        'elec_rate', 'elec_fixed', 'elec_CO2_rate'
                                       ]]
    ## assign rates
    p['gas_rate'] = p['building_id'].map(res['gas_rate'])
    p['gas_fixed'] = p['building_id'].map(res['gas_fixed'])
    p['gas_CO2_rate'] = p['building_id'].map(res['gas_CO2_rate'])
    p['elec_rate'] = p['building_id'].map(res['elec_rate'])
    p['elec_fixed'] = p['building_id'].map(res['elec_fixed'])
    p['elec_CO2_rate'] = p['building_id'].map(res['elec_CO2_rate'])
    
    if not for_packages:
        # ind upgrades
        for n in HVAC_upgrades_rate_change.keys():
            new_rates = HVAC_upgrades_rate_change[n]
            p.loc[p['simulation_output_report.applicable']==True, 'gas_rate'] = new_rates[0]
            p.loc[p['simulation_output_report.applicable']==True, 'gas_fixed'] = new_rates[1]
            p.loc[p['simulation_output_report.applicable']==True, 'elec_rate'] = new_rates[2]
            p.loc[p['simulation_output_report.applicable']==True, 'elec_fixed'] = new_rates[3]
    else:
        if isinstance(HVAC_upgrades_rate_change, dict):
            # packages with a dict input
            if upkg_no in HVAC_upgrades_rate_change.keys():
                for m in HVAC_upgrades_rate_change[upkg_no]:
                    idx = (p[(p['apply_upgrade.applicable']==True) &
                           (~p[f'simulation_output_report.option_{m:02d}_cost_usd'].isnull())
                            ]).index
                    p.loc[idx, 'gas_rate'] = NGH_rate
                    p.loc[idx, 'gas_fixed'] = NGH_fixed
                    p.loc[idx, 'elec_rate'] = EH_rate
                    p.loc[idx, 'elec_fixed'] = EH_fixed
        
        else:
            # packages with a list input
            for m in HVAC_upgrades_rate_change:
                idx = (p[(p['apply_upgrade.applicable']==True) &
                       (~p[f'simulation_output_report.option_{m:02d}_cost_usd'].isnull())
                        ]).index
                p.loc[idx, 'gas_rate'] = NGH_rate
                p.loc[idx, 'gas_fixed'] = NGH_fixed
                p.loc[idx, 'elec_rate'] = EH_rate
                p.loc[idx, 'elec_fixed'] = EH_fixed
                
    # assign 0 rates to building with no energy use  
    p.loc[p['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_fixed'] = 0 
    p.loc[p['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_rate'] = 0 
    p.loc[p['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_fixed'] = 0 
    p.loc[p['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_rate'] = 0 
    
    for col in ['gas_rate','gas_fixed','gas_CO2_rate','elec_rate','elec_fixed','elec_CO2_rate']:
        p.loc[p['simulation_output_report.applicable']==False, col] = np.nan

    return p

def combine_upgrade_cost_and_lifetime(p):
    
    ## upgrade costs (sum)
    p['upgrade_cost'] = p['simulation_output_report.upgrade_cost_usd']
    cost_cols = list(x for x in p.columns if x.endswith('cost_usd'))
    p = p.drop(cost_cols, axis=1)
    
    ## upgrade lifetime (min)
    lt_cols = list(x for x in p.columns if x.endswith('lifetime_yrs'))
    p['upgrade_lifetime'] = p[lt_cols].min(axis=1)
    p = p.drop(lt_cols, axis=1)

    p['upgrade_cost'] = p['upgrade_cost'].replace([0, None,''],np.nan)
    p['upgrade_lifetime'] = p['upgrade_lifetime'].replace([0, None,''],np.nan)
    
    return p

def get_annual_totals(pp, get_col_only=False):
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
        
    p['ann_gas_cost'] = \
        p['simulation_output_report.total_site_natural_gas_therm']*p['gas_rate']+p['gas_fixed']
    p['ann_elec_cost'] = \
        p['simulation_output_report.total_site_electricity_kwh']*p['elec_rate']+p['elec_fixed']
    p['ann_energy_cost'] = \
        p['ann_gas_cost'] + p['ann_elec_cost']
    
    p['ann_metric_ton_co2e_gas'] = \
        p['simulation_output_report.total_site_natural_gas_therm']*p['gas_CO2_rate']
    p['ann_metric_ton_co2e_elec'] = \
        p['simulation_output_report.total_site_electricity_kwh']*p['elec_CO2_rate']
    p['ann_metric_ton_co2e'] = \
        p['ann_metric_ton_co2e_gas'] + p['ann_metric_ton_co2e_elec']
    
    if get_col_only:
        return p[['ann_gas_cost','ann_elec_cost','ann_energy_cost',
                 'ann_metric_ton_co2e_gas','ann_metric_ton_co2e_elec','ann_metric_ton_co2e']]
    else:
        return p

def get_annual_gas_elec_site_energy_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_therm_gas_saving'] = ref['simulation_output_report.total_site_natural_gas_therm']-\
         p['simulation_output_report.total_site_natural_gas_therm']
    p['ann_kwh_elec_saving'] = ref['simulation_output_report.total_site_electricity_kwh']-\
         p['simulation_output_report.total_site_electricity_kwh']
    p['ann_mbtu_site_energy_saving'] = ref['simulation_output_report.total_site_energy_mbtu']-\
        p['simulation_output_report.total_site_energy_mbtu']
    
    if get_col_only:
        return p[['ann_therm_gas_saving','ann_kwh_elec_saving','ann_mbtu_site_energy_saving']]
    else:
        return p
    
def get_thermal_saving_breakdown(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    # gas thermal components:
    p['gas_heating_mbtu_saving'] = (ref['simulation_output_report.natural_gas_heating_therm']-\
         p['simulation_output_report.natural_gas_heating_therm'])*therm_to_mbtu
    p['gas_hot_water_mbtu_saving'] = (ref['simulation_output_report.natural_gas_water_systems_therm']-\
         p['simulation_output_report.natural_gas_water_systems_therm'])*therm_to_mbtu
    
    # electric thermal components:
    for eu in thermal_eu_main:
        eu_list = list(thermal_eu_df.query('enduse == @eu').index)
        p[f'elec_{eu}_mbtu_saving'] = (ref[eu_list].sum(axis=1)-p[eu_list].sum(axis=1))*kwh_to_mbtu
    
    # other thermal components:
    other_heating_eu = [
        'simulation_output_report.fuel_oil_heating_mbtu',
        'simulation_output_report.propane_heating_mbtu',
        'simulation_output_report.wood_heating_mbtu',
    ]
    p['other_heating_mbtu_saving'] = ref[other_heating_eu].sum(axis=1)-p[other_heating_eu].sum(axis=1)
    
    other_hot_water_eu = [
        'simulation_output_report.fuel_oil_water_systems_mbtu',
        'simulation_output_report.propane_water_systems_mbtu',
    ]
    p['other_hot_water_mbtu_saving'] = ref[other_hot_water_eu].sum(axis=1)-p[other_hot_water_eu].sum(axis=1)
    
    if get_col_only:
        return p[['gas_heating_mbtu_saving','gas_hot_water_mbtu_saving',
                  'elec_heating_mbtu_saving','elec_cooling_mbtu_saving','elec_hvac_fan_pump_mbtu_saving','elec_hot_water_mbtu_saving',
                  'other_heating_mbtu_saving','other_hot_water_mbtu_saving',
                 ]]
    else:
        return p

def get_annual_energy_cost_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """ 
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_gas_cost_saving'] = \
        ref['simulation_output_report.total_site_natural_gas_therm']*ref['gas_rate']+ref['gas_fixed'] - \
        (p['simulation_output_report.total_site_natural_gas_therm']*p['gas_rate']+p['gas_fixed'])
        
    p['ann_elec_cost_saving'] = \
        ref['simulation_output_report.total_site_electricity_kwh']*ref['elec_rate']+ref['elec_fixed'] - \
        (p['simulation_output_report.total_site_electricity_kwh']*p['elec_rate']+p['elec_fixed'])
    
    p['ann_energy_cost_saving'] = p['ann_gas_cost_saving']+p['ann_elec_cost_saving'] #p[['ann_gas_cost_saving','ann_elec_cost_saving']].sum(axis=1)
    
    if get_col_only:
        return p['ann_energy_cost_saving']
    else:
        return p

def get_annual_metric_ton_co2e_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_metric_ton_co2e_saving_gas'] = \
        (ref['simulation_output_report.total_site_natural_gas_therm']-\
         p['simulation_output_report.total_site_natural_gas_therm'])*p['gas_CO2_rate']
    
    p['ann_metric_ton_co2e_saving_elec'] = \
        (ref['simulation_output_report.total_site_electricity_kwh']-\
         p['simulation_output_report.total_site_electricity_kwh'])*p['elec_CO2_rate']
    
    p['ann_metric_ton_co2e_saving'] = p['ann_metric_ton_co2e_saving_gas'] + p['ann_metric_ton_co2e_saving_elec']
    
    if get_col_only:
        return p['ann_metric_ton_co2e_saving']
    else:
        return p

def print_metrics_report(p, has_comparative_payback=True):
    ### (1) check for simple_payback > 100 yr or if < 0 yr
    if len(p[~p['simple_payback'].isnull()]):
        print(p['simple_payback'].agg(['min','median','max']))
    spb_100 = p[p['simple_payback']>100]
    if len(spb_100)>0:
        print(f'    *PAYBACK1 - too large* upgrade={n} has {len(spb_100)} simple_payback>100 ' +\
              f'(including {len(p[p["simple_payback"]==np.inf])} INF)')
    spb_neg = p[p['simple_payback']<0]
    if len(spb_neg)>0:
        print(f'    *PAYBACK1 - negative*  upgrade={n} has {len(spb_neg)} simple_payback<0 ' +\
              '(due to negative energy cost saving)')

    ### (2) check for comparative_payback > 100 yr or if < 0 yr
    if has_comparative_payback:
        if len(p[~p['comparative_payback'].isnull()]):
            print(p['comparative_payback'].agg(['min','median','max']))
        spb_100 = p[p['comparative_payback']>100]
        if len(spb_100)>0:
            print(f'    *PAYBACK2 - too large* upgrade={n} has {len(spb_100)} comparative_payback>100 ' +\
                  f'(including {len(p[p["comparative_payback"]==np.inf])} INF)')
        spb_neg = p[p['comparative_payback']<0]
        if len(spb_neg)>0:
            print(f'    *PAYBACK2 - negative*  upgrade={n} has {len(spb_neg)} comparative_payback<0 ' +\
                  '(due to negative energy cost saving)')

    ### (3) check for eui==inf
    for eui in ['pct_delta_gas_eui','pct_delta_elec_eui','pct_delta_site_eui']:
        eui_inf = p[p[eui]==np.inf]
        if len(eui_inf)>0:
            print(f'       *EUI - inf* upgrade={n} has {len(eui_inf)} {eui}=INF ' +\
                  '(due to fuel introduction from upgrade)')

    ### (4) check for neg carbon savings
    ces_neg = p[p['ann_metric_ton_co2e_saving']<0]
    if len(ces_neg)>0:
        print(f'    *CARBON - negative*  upgrade={n} has {len(ces_neg)} carbon saving<0 ')
        
print('funcs loaded')

In [ ]:
# ### set utility rates ###
# NG_rate_multiplier = 1 # <-----

# if NG_rate_multiplier > 1:
#     fn_ext = f'_{NG_rate_multiplier}x_gas_prices' # file name extension to add to relevant results
# else:
#     fn_ext = ''

# ### utility rates ###########################################
# # ref (EIA): 
# # avg ComEd res elec rate 2019: $ 0.1330 /kWh
# # weighted avg IL gas rate 2019: $ 0.77183 /therm

# ## electricity ##
# # annual fixed rates = monthly x 12
# EH_fixed = 15.70 * 12 # annual
# NEH_fixed = 14.28 *12 # annual
# # avg of summer rates (J,J,A,S) and non-summer rates
# EH_rate = (0.10273*4+0.10952*8)/12 # 0.08019, $/kWh, electric rate for electric heating customers
# NEH_rate = (0.12168*4+0.12847*8)/12 # 0.09889, $/kWh, electric rate for non-electric heating customers
# # marginal carbon emission factor:
# elec_CO2_rate = 0.000834702 # metric tons of CO2e/kWh (0.2446 tons/mbtu)

# ## gas ##
# # annual fixed rates = monthly x 12
# GH_fixed = 45.32 * 12
# NGH_fixed = 21.51 * 12
# # variable rates
# GH_rate = 0.61648 * NG_rate_multiplier # 0.19477, $/therm, gas rate for NG heating customers
# NGH_rate = 0.56758 * NG_rate_multiplier # 0.14964, $/therm, gas rate for non-NG heating customers
# # marginal carbon emission factor:
# gas_CO2_rate = 0.00532181 # metric tons of CO2e/therm (0.0532 tons/mbtu)

# ### upgrades that will cause utility rate change: ###########################################
# HVAC_upgrades_rate_change = {
#     17: [4,6,7], # ASHP
#     18: [4,6,7,8], # MSHP,
# }

# # assign rates accordingly
# Elec_heating_types = ['Electricity Baseboard','Electricity ASHP','Electricity Electric Furnace',
#                       'Electricity Electric Boiler', 'Electricity Electric Wall Furnace']
# NG_heating_types = ['Natural Gas Fuel Wall/Floor Furnace', 'Natural Gas Fuel Furnace',
#                     'Natural Gas Fuel Boiler']

# res['gas_rate'] = NGH_rate
# res['gas_fixed'] = NGH_fixed
# res['gas_CO2_rate'] = gas_CO2_rate
# res['elec_rate'] = NEH_rate
# res['elec_fixed'] = NEH_fixed
# res['elec_CO2_rate'] = elec_CO2_rate

# res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(NG_heating_types)]).index,
#        'gas_rate'] = GH_rate
# res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(NG_heating_types)]).index,
#        'gas_fixed'] = GH_fixed
# res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(Elec_heating_types)]).index,
#         'elec_rate'] = EH_rate
# res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(Elec_heating_types)]).index,
#         'elec_fixed'] = EH_fixed

# # assign 0 gas rates to building with no gas use 
# res.loc[res['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_fixed'] = 0 
# res.loc[res['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_rate'] = 0 
# res.loc[res['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_fixed'] = 0 
# res.loc[res['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_rate'] = 0

# for col in ['gas_rate','gas_fixed','gas_CO2_rate','elec_rate','elec_fixed','elec_CO2_rate']:
#     res.loc[res['completed_status']!='Success', col] = np.nan

# print(f'Natural gas rate multiplier: {NG_rate_multiplier}')


### Get baseline results

In [ ]:
#### count upgrades
N_upgrades = nupgrades_dict[pkg_version] # <--

print(f'>>> {pkg_version} has {N_upgrades:,} packages...')
    
## get summary table
summary_upgrades = []

## initialize dB table
# all_proto_upgrades = []

package_list = range(1,1+N_upgrades)
for n in package_list:
    p = pd.read_csv(result_path / f'results_up{n:02d}.csv')
    print(f'\nPackage {n}')
    
    upgrade_name = p['apply_upgrade.upgrade_name'].replace('',np.nan).dropna(axis=0).unique()[0]
    ### assign utility rates
#     p = assign_utility_rates_to_upgrade(n, p, res, HVAC_upgrades_rate_change, for_packages=True)

    if upgrade_name in pkg_to_fix:
        p = fix_upgrade_costs(p, res)
    
    ### collapse upgrade cost and lifetime cols
    p = combine_upgrade_cost_and_lifetime(p)

    ### check if upgrade has 0 successful sims
    if len(p[p['completed_status']=='Success']) == 0:
        print(f' * upgrade={n} has 0 successful simulations')

    ### calculate metrics
    p = add_thermal_load_sqft_eui(p, res)
    EUIi = ['gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_mbtupersqft']
    EUIo = ['gas_eui','elec_eui','site_eui']
    for vari, varo in zip(EUIi, EUIo):
        p[f'pct_delta_{varo}'] = ((p[vari]-res[vari])/res[vari]*100)


    # annual energy saving:
    p = get_annual_gas_elec_site_energy_saving(p, res)
    p = get_thermal_saving_breakdown(p, res)

    # annual energy cost saving:
#     p = get_annual_energy_cost_saving(p, res)
    
    # annual kBtu saved per upgrade cost:
#     p['ann_kbtu_saved_per_dollar'] = p['ann_mbtu_site_energy_saving'].divide(
#                             p['upgrade_cost'], axis=0)*1000

    # simple payback
#     p['simple_payback'] = p['upgrade_cost']/p['ann_energy_cost_saving']

    # annual metric ton carbon emission savings:
#     p = get_annual_metric_ton_co2e_saving(p, res)

    ### check for neg/large paybacks, inf eui, neg carbon savings 
#     print_metrics_report(p, has_comparative_payback=False)
    
    ### subset to only those that have been applied with the upgrades successfully 
    # and also have a successful baseline:
    upgrade_name_short = upgrade_rename_dict[upgrade_name]
    p = p[(p['apply_upgrade.applicable']==True) & (res['completed_status']=='Success')].reset_index(drop=True)
    p['package_no'] = n
    
    # clean up upgrade name as needed
    if upgrade_name in upgrade_name_correction_dict:
        p['upgrade_name'] = upgrade_name_correction_dict[upgrade_name] 
    else:
        p['upgrade_name'] = upgrade_name
    p['upgrade_name_short'] = upgrade_name_short
    
    ### add to dB
#     all_proto_upgrades.append(p)
    
    # save processed results
    p[p['completed_status']=='Success'].to_csv(resdir / f'results_up{n:02d}.csv', index=False)
    
    for eui in ['pct_delta_gas_eui','pct_delta_elec_eui','pct_delta_site_eui']:
        p[eui] = p[eui].replace([np.inf, -np.inf], np.nan) # for mean calc
        
    ### add to summary table
    summ = pd.DataFrame(index=[0])
    level0 = ''
    summ[(level0, 'upgrade_no')] = n
    summ[(level0, 'upgrade_name')] = upgrade_name
    summ[(level0, 'upgrade_name_short')] = upgrade_name_short
    summ[(level0, 'sample_weight')] = res['build_existing_model.sample_weight'].mean()
    summ[(level0, 'n_applied')] = len(p)
    summ[(level0, 'n_success')] = len(p[p['completed_status']=='Success'])
    summ[(level0, 'n_fail')] = len(p[p['completed_status']=='Fail'])
    summ[(level0, 'n_invalid')] = len(p[p['completed_status']=='Invalid'])
    summ[(level0, 'pct_success')] = round(summ[(level0,'n_success')]/summ[(level0,'n_applied')]*100,3)
    summ.columns = pd.MultiIndex.from_tuples(summ.columns)

    para = ['mean','std', 'min','50%','max']
    p = p[p['completed_status']=='Success'].reset_index(drop=True)
    
    ## (0) Upgrade cost
    summ = pd.concat([
        summ,
        p['upgrade_cost'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    
    ## (1) Baselines
    consum = pd.DataFrame(index=[0])
    level1 = 'baseline - mean'
    # 1.1. Annual Totals
    consum[(level1, 'total gas (therm)')] = round(
        res.loc[p.index, 'simulation_output_report.total_site_natural_gas_therm'].mean(), 
        3)
    consum[(level1, 'total elec (kwh)')] = round(
        res.loc[p.index, 'simulation_output_report.total_site_electricity_kwh'].mean(), 
        3)
    consum[(level1, 'total site (mbtu)')] = round(
        res.loc[p.index, 'simulation_output_report.total_site_energy_mbtu'].mean(), 
        3)

    # 1.2. Thermal Totals
    consum[(level1, 'thermal gas (therm)')] = round(
        res.loc[p.index, 'thermal_gas_therm'].mean(), 
        3)
    consum[(level1, 'thermal elec (kwh)')] = round(
        res.loc[p.index, 'thermal_elec_kwh'].mean(), 
        3)
    consum[(level1, 'thermal site (mbtu)')] = round(
        res.loc[p.index, 'thermal_site_mbtu'].mean(), 
        3)
    
    # 1.3. Thermal EUIs
    consum[(level1, 'thermal_gas_eui (therm/sqft)')] = round(
        res.loc[p.index, 'gas_eui_thermpersqft'].mean(), 
        3)
    consum[(level1, 'thermal_elec_eui (kwh/sqft)')] = round(
        res.loc[p.index, 'elec_eui_kwhpersqft'].mean(), 
        3)
    consum[(level1, 'thermal_site_eui (mbtu/sqft)')] = round(
        res.loc[p.index, 'site_eui_mbtupersqft'].mean(), 
        3)
    
    ## (2) Upgrades
    level2 = 'upgrade - mean'
    # 2.1. Annual Totals
    consum[(level2, 'gas (therm)')] = round(p['simulation_output_report.total_site_natural_gas_therm'].mean(), 3)
    consum[(level2, 'elec (kwh)')] = round(p['simulation_output_report.total_site_electricity_kwh'].mean(), 3)
    consum[(level2, 'site (mbtu)')] = round(p['simulation_output_report.total_site_energy_mbtu'].mean(), 3)
    
    # 2.2. EUIs
    consum[(level2, 'gas_eui (therm/sqft)')] = round(p['gas_eui_thermpersqft'].mean(), 3)
    consum[(level2, 'elec_eui (kwh/sqft)')] = round(p['elec_eui_kwhpersqft'].mean(), 3)
    consum[(level2, 'site_eui (mbtu/sqft)')] = round(p['site_eui_mbtupersqft'].mean(), 3)
    
    consum.columns = pd.MultiIndex.from_tuples(consum.columns)
    summ = pd.concat([summ, consum], axis=1)
    
    
    ## (3-1) Savings ##
    level3 = 'savings'
    # 3-1.1 Annual Totals
    summ = pd.concat([
        summ,
        p['ann_therm_gas_saving'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    summ = pd.concat([
        summ,
        p['ann_kwh_elec_saving'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    summ = pd.concat([
        summ,
        p['ann_mbtu_site_energy_saving'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    
    summ = pd.concat([
        summ,
        p['pct_delta_gas_eui'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    summ = pd.concat([
        summ,
        p['pct_delta_elec_eui'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    summ = pd.concat([
        summ,
        p['pct_delta_site_eui'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    

    
    summary_upgrades.append(summ)
    
summary_upgrades = pd.concat(summary_upgrades).reset_index(drop=True)
summary_upgrades


In [ ]:
### export UPGRADE summary
summary_upgrades.to_csv(outdir / f'upgrades_summary.csv', index=False)
print(f'UPGRADE summary table saved')